# 人脸检测

## 1.导入所需模块

In [ ]:
from nxbot import Robot,ObjectDetector,bgr8_to_jpeg,event,pid
from IPython.display import display
import ipywidgets.widgets as widgets
import cv2
import threading
import time

## 2.加载opencv人脸识别模型

In [ ]:
# 加载opencv人脸检测所需文件
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# 加载opencv眼睛检测所需文件
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

## 3.小车桌面控制按钮与摄像头控制滑块

In [ ]:
import control
controls_box = control.controls_box
camera_slider = control.camera_slider

## 4.初始化参数

In [ ]:
# 初始化人脸坐标与尺寸
global face_x, face_y, face_w, face_h
face_x = face_y = face_w = face_h = 0

# 设置PID参数
xservo_pid = pid.PositionalPID(2, 0.1, 1)
yservo_pid = pid.PositionalPID(2, 0.1, 1)

# 创建显示部件
image_widget = widgets.Image(format='jpeg', width=300, height=300)
speed_widget = widgets.FloatSlider(value=0.0, min=0.0, step=0.01, max=0.5, description='运行速度')
turn_gain_widget = widgets.FloatSlider(value=2.0, min=0.0, step=0.01, max=5.0, description='转向增益')

# 设置任务类型
task1 ='camera_following'
task2 ='robot_following'
task = task2

## 5.开始进行人脸识别并执行相应的任务

In [ ]:
def _prediction():
    global detect_face
    detect_face = True
    while detect_face:    
        time.sleep(0.001)
        img = rbt.camera.read()
        if img is not None:
            newImage = cv2.resize(img, (300, 300))
            # 将图片变为灰度图
            gray_img = cv2.cvtColor(newImage,cv2.COLOR_BGR2GRAY)
            # 图片输入人脸检测模型
            faces = face_cascade.detectMultiScale(gray_img, scaleFactor = 1.1,minNeighbors = 3, minSize = (10,10),flags = cv2.CASCADE_SCALE_IMAGE)
            # 如果检测到人脸
            if len(faces)>0:
                # 人脸坐标
                (face_x, face_y, face_w, face_h) = faces[0]
                # 将人脸在图片上框出来
                cv2.rectangle(img,(face_x+10,face_y),(face_x+face_w-10,face_y+face_h+20),(0,255,0),2)
                # 人脸中心点坐标
                face_center = [face_x+face_w/2.0, face_y+face_h/2.0]
                # 如果任务为摄像头跟踪
                if task=='camera_following':
                    #Proportion-Integration-Differentiation
                    xservo_pid.SystemOutput = face_center[0]
                    xservo_pid.SetStepSignal(150)
                    xservo_pid.SetInertiaTime(0.01, 0.06)
                    target_valuex = int(xservo_pid.SystemOutput)
                    # 输入Y轴方向参数PID控制输入
                    yservo_pid.SystemOutput = face_center[1]
                    yservo_pid.SetStepSignal(150)
                    yservo_pid.SetInertiaTime(0.01, 0.06)
                    target_valuey = int(yservo_pid.SystemOutput)
                         # 将云台转动至PID调校位置
                    rbt.base.set_ptz(target_valuey/5,-target_valuex/5)

                # 如果任务为机器人跟踪
                elif task=='robot_following':
                    steering = (150 - face_center[0])/150/2*turn_gain_widget.value
                    xservo_pid.SystemOutput = steering
                    xservo_pid.SetStepSignal(150)
                    xservo_pid.SetInertiaTime(0.01, 0.006)
                    target_steering = int(xservo_pid.SystemOutput)
                    rbt.base.move(speed_widget.value, 0, target_steering)

            else:
                rbt.base.move(0, 0, 0)
                
            image_widget.value = bgr8_to_jpeg(img)
            

## 6.连接小车并进行人脸检测

In [ ]:
rbt = control.rbt
rbt.connect()
rbt.camera.start()
rbt.base.set_ptz(0)
process1 = threading.Thread(target=_prediction,)
process1.start()

display(widgets.HBox([image_widget, widgets.VBox([speed_widget,turn_gain_widget]), controls_box]))
display(camera_slider)

## 7.断开与小车连接

In [ ]:
# detect_face=False
# control.unlink_control()
# rbt.disconnect()